In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
print(torch.__version__)

1.13.1+cu117


In [2]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [30]:
set_seed()
linear = nn.Linear(2, 1)
model_params = list(linear.parameters())  # returns weights and biases
print('Perceptron Weights : ', model_params[0][0].data.numpy())
print('Perceptron Bias :', model_params[1].data.numpy())
x = torch.tensor(0.0)
torch.sigmoid(x)

Perceptron Weights :  [-0.00529398  0.3793229 ]
Perceptron Bias : [-0.58198076]


tensor(0.5000)

In [4]:
class XOR(nn.Module):
    def __init__(self):
        super(XOR, self).__init__()
        self.linear = nn.Linear(2, 2)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(2, 1)

    def forward(self, input):
        x = self.linear(input)
        sig = self.sigmoid(x)
        yh = self.linear2(sig)
        return yh
